Script to generate and push MeasureReport resources

In [1]:
import json
import requests
from fhirpathpy import evaluate, compile
from copy import deepcopy

In [2]:
base_url = "https://fhir.agg-data.connectathon.dev.cirg.uw.edu"
fhir_endpoint = f'{base_url}/fhir'
resource_type = "MeasureReport"
search_params = {
    "date": "ge2020-01-01",
    "_count": "1000",
}
headers = {
    "Content-Type": "application/json+fhir"
}
full_url = f'{fhir_endpoint}/{resource_type}'

In [3]:
cred_manager = None

In [164]:
# phdi attempt: SLOW performance, moved to requests library
# entries = []
# tmp_search_url = full_search_url
# while tmp_search_url is not None:
#     (page_entries, search_url) = extract_data_from_fhir_search_incremental(search_url=tmp_search_url, cred_manager=cred_manager)
#     entries.append(page_entries)


In [4]:
entries = []
url = full_url

get_entries = compile("Bundle.entry")

next_url_exists = compile("Bundle.link.where(relation='next').exists()")
check_next_url = lambda x: next_url_exists(x)[0]

next_url_path = compile("Bundle.link.where(relation='next').url")
get_next_url = lambda x: next_url_path(x)[0]

while True:
    if (url == full_url):
        response = requests.get(url=full_url, params=search_params, headers=headers)
    else:
        response = requests.get(url=url, headers=headers)
    content = json.loads(response.content)
    entries.extend(get_entries(content))
    #update the url
    if check_next_url(content):
        url = get_next_url(content)
        print(url)
    else:
        break

https://fhir.agg-data.connectathon.dev.cirg.uw.edu/fhir?_getpages=909bbbff-12ed-462b-9bee-516d344b691a&_getpagesoffset=1000&_count=1000&_pretty=true&_bundletype=searchset
https://fhir.agg-data.connectathon.dev.cirg.uw.edu/fhir?_getpages=909bbbff-12ed-462b-9bee-516d344b691a&_getpagesoffset=2000&_count=1000&_pretty=true&_bundletype=searchset


In [5]:
print(response.status_code)
if response.status_code >= 400:
        print(response.text)

200


In [6]:
len(entries)

2715

In [7]:
measureSet = set()
for entry in entries:
    measureSet.add(entry['resource']['measure'])
list(measureSet)

['http://hl7.org/fhir/uv/saner/Measure/CDCPatientImpactAndHospitalCapacity',
 'https://cqf-ruler.ecqm.icfcloud.com/fhir/Measure/ICUBedsReporting',
 'https://cqf-ruler.ecqm.icfcloud.com/fhir/Measure/ICUBedStaffedCapacity',
 'http://hl7.org/fhir/uv/saner/Measure/FEMADailyHospitalCOVID19Reporting',
 'https://cqf-ruler.ecqm.icfcloud.com/fhir/Measure/ICUBedCurrentOccupancy']

In [204]:
measures_to_replace = [ ]
#     'ICUBedStaffedCapacity',
#     'ICUBedCurrentOccupancy',
#     'ICUBedsReporting'
# ]
updateBundle = {
    "resourceType": "Bundle",
	"id": "mr-update",
	"type": "transaction",
	"entry": []
}
measure_report_measure = compile("MeasureReport.measure")
get_measure = lambda x: measure_report_measure(x)[0]

for entry in entries:
    measure = str.split(get_measure(entry['resource']), "/")[-1]
    if get_measure(entry['resource']) in measures_to_replace:
        updateEntry = deepcopy(entry)
        updateEntry['resource']['measure'] = 'https://cqf-ruler.ecqm.icfcloud.com/fhir/Measure/' + measure
        updateEntry.pop('search', None)
        updateEntry['request'] = {
            "method": "PUT",
            "url": f"{updateEntry['resource']['resourceType']}/{updateEntry['resource']['id']}"
        }
        updateBundle['entry'].append(updateEntry)

In [205]:
len(updateBundle['entry'])

247

In [206]:
updateBundle

{'resourceType': 'Bundle',
 'id': 'mr-update',
 'type': 'transaction',
 'entry': [{'fullUrl': 'https://fhir.agg-data.connectathon.dev.cirg.uw.edu/fhir/MeasureReport/1761',
   'resource': {'resourceType': 'MeasureReport',
    'id': '1761',
    'meta': {'versionId': '3',
     'lastUpdated': '2023-05-10T19:43:16.233+00:00',
     'source': '#osFVJFbF66oTDCdk',
     'profile': ['http://hl7.org/fhir/uv/saner/StructureDefinition/PublicHealthMeasureReport']},
    'contained': [{'resourceType': 'Organization',
      'id': '589723',
      'name': 'Baseline West Medical Center1'},
     {'resourceType': 'Location',
      'id': '494e0440-ec4d-11ed-af8b-cfdfdf3415fa',
      'name': 'Helios FHIR Accelerator for Public Health'}],
    'identifier': [{'system': 'urn:ietf:rfc:3986',
      'value': 'urn:uuid:cd699038-9422-0ff8-2c9c-fac3147a420d'}],
    'status': 'complete',
    'type': 'summary',
    'measure': 'https://cqf-ruler.ecqm.icfcloud.com/fhir/Measure/ICUBedsReporting',
    'subject': {'reference

In [207]:
if len(updateBundle['entry']) > 0:
    url = base_url
    hed = {'Content-Type': 'application/json+fhir'}
    data = json.dumps(updateBundle)

    response = requests.post(url=url, data=data, headers=hed)
    print(response.text)
    print(response.status_code)

{
  "resourceType": "Bundle",
  "id": "29ee2539-8a15-4098-a443-e2364d2276d6",
  "type": "transaction-response",
  "link": [ {
    "relation": "self",
    "url": "https://fhir.agg-data.connectathon.dev.cirg.uw.edu/fhir"
  } ],
  "entry": [ {
    "response": {
      "status": "200 OK",
      "location": "MeasureReport/1761/_history/4",
      "etag": "4",
      "lastModified": "2023-05-10T19:43:16.233+00:00"
    }
  }, {
    "response": {
      "status": "200 OK",
      "location": "MeasureReport/1763/_history/4",
      "etag": "4",
      "lastModified": "2023-05-10T19:43:16.233+00:00"
    }
  }, {
    "response": {
      "status": "200 OK",
      "location": "MeasureReport/1764/_history/4",
      "etag": "4",
      "lastModified": "2023-05-10T19:43:16.233+00:00"
    }
  }, {
    "response": {
      "status": "200 OK",
      "location": "MeasureReport/1765/_history/4",
      "etag": "4",
      "lastModified": "2023-05-10T19:43:16.233+00:00"
    }
  }, {
    "response": {
      "status": "